In [ ]:
import importlib        # for refreshing imports
import pickle           # to load models
import requests         # to make an API request and get recent data
import pandas as pd     # for data
import numpy as np      # for math
import settings         # for settings

# reloads settings to get recent info
importlib.reload(settings)

# class for invalid API Key exception
class APIKeyException(Exception): 
    def __init__(self, message):
        self.message = message  # custom error message for many types of errors (api key not found, api key invalid, rate limited, etc)

In [ ]:
# checks for API key
if not settings.APIKEY:
    raise APIKeyException("Please enter an API key!")   

# loads model and scaler
with open(f"{settings.IPYNBMODELPATH}model.pkl", "rb") as f:
    model = pickle.load(f)
with open(f"{settings.IPYNBMODELPATH}scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

In [ ]:
# returns percent change 
def calcPercentChange(open, close):
    return (float(close)) / float(open)

# gets stock
symbol = input("Enter your stock symbol which you are analyzing: ")

# gets the information for the stock
url = "https://www.alphavantage.co/query"
params = {
    "function": "TIME_SERIES_DAILY_ADJUSTED",
    "apikey": settings.APIKEY,
    "symbol": symbol,
    "datatype": "json"
}
data = requests.get(url, params=params).json()

# try to look at data otherwise api key is limited
try:
    data = data["Time Series (Daily)"]
except KeyError:
    try: 
        data["Error Message"]       # checks if there is an error message
        raise APIKeyException("Your API key is invalid, or you entered an invalid stock.")
    except KeyError:                         # otherwise it must be a rate limit
        raise APIKeyException("You have been rate limited. Try again later.")

# gets most recent data and reverses it
data = list(data.values())[:int(settings.DATACOUNT * (1-settings.CLASSIFYSPLIT)):][::-1]  # TURNS THE DATA INTO THE MOST RECENT TRADING DAYS

# gets array of percent changes in the stock
gains = [calcPercentChange(open=day["1. open"], close=day["4. close"]) for day in data]

frame = pd.DataFrame({      # PARAMETERS HERE SHOULD MATCH THOSE IN train.py
    "gainVariation": [np.std(gains)],
    "avgDailyIncrease": [np.average(gains)],
    "overallIncrease": [calcPercentChange(open=data[0]["4. close"], close=data[-1]["4. close"])],
    "nDayIncrease": [calcPercentChange(open=data[-settings.NDAYS]["4. close"], close=data[-1]["4. close"])]
})

# scales dataframe
frame = scaler.transform(frame)

# predicts outcome
outcome = list(model.predict(frame))[0]
probability = list(model.predict_proba(frame))[0][outcome]

# prints output
print(f"You should {['sell', 'buy'][outcome]} {symbol} with a certainty of {round(probability, 3)}.")
print(f"The model predicts {['a decrease', 'an increase'][outcome]} over the next {int(settings.CLASSIFYSPLIT * settings.DATACOUNT)} days.")